# Trace Visualization Demo

This notebook demonstrates minimal examples for each EventType with the integrated `deps` parameter for variable dependency tracking.

In [ ]:
# Import required modules
import sys
import os

# Add pywhy to path if needed
sys.path.append('..')

from pywhy.trace_visualization import create_jupyter_trace_display
from pywhy.trace_dsl import trace
from pywhy.instrumenter import exec_instrumented
from pywhy.tracer import get_tracer
from IPython.display import HTML, display

print("Modules imported successfully!")

In [ ]:
# Helper function for trace comparison
def compare(source_code, expected_trace, title="Trace Comparison"):
    """Compare actual execution trace with expected trace"""
    tracer = get_tracer()
    tracer.clear()
    
    exec_instrumented(source_code)
    actual_events = tracer.events
    
    html_output = create_jupyter_trace_display(actual_events, expected_trace, title)
    display(HTML(html_output))
    
    tracer.clear()

print("Helper function defined!")

## 1. ASSIGN - Variable Assignment

Simple variable assignment with dependency tracking.

In [ ]:
# Create expected trace using DSL
source_code = """
x = 10
y = 20
z = x + y
"""

expected_trace = (
    trace()
    .assign("x", 10)
    .assign("y", 20) 
    .assign("z", 30, deps=['x', 'y'])
    .build()
)

compare(source_code, expected_trace, "ASSIGN Event")

## 2. AUG_ASSIGN - Augmented Assignment

Augmented assignments (+=, -=, *=, etc.) with dependency tracking.

In [ ]:
# Create expected trace using DSL
source_code = """
x = 10
y = 5
x += y
"""

expected_trace = (
    trace()
    .assign("x", 10)
    .assign("y", 5)
    .assign("x", 15, "aug", deps=['x', 'y'])  # x += y reads both x and y
    .build()
)

compare(source_code, expected_trace, "AUG_ASSIGN Event")

## 3. BRANCH - Conditional Branches

Branch events for if/else statements with condition dependency tracking.

In [ ]:
# Create expected trace using DSL
source_code = """
x = 10
if x > 5:
    y = 100
else:
    y = 200
"""

expected_trace = (
    trace()
    .assign("x", 10)
    .branch("x > 5", True, "if_block", deps=['x'])
    .assign("y", 100)
    .build()
)

compare(source_code, expected_trace, "BRANCH Event")

## 4. FUNCTION_ENTRY and RETURN - Function Calls

Function entry and return events.

In [ ]:
# Create expected trace using DSL
source_code = """
def add(a, b):
    return a + b

x = 3
result = add(x, 4)
"""

expected_trace = (
    trace()
    .assign("x", 3)
    .function_entry("add", [3, 4])
    .return_event(7)
    .assign("result", 7, deps=['add', 'x'])
    .build()
)

compare(source_code, expected_trace, "FUNCTION_ENTRY and RETURN Events")

## 5. LOOP_ITERATION - For Loops

Loop iteration events for for loops.

In [ ]:
# Create expected trace using DSL
source_code = """
total = 0
for i in [1, 2, 3]:
    total += i
"""

expected_trace = (
    trace()
    .assign("total", 0)
    .loop_iteration("i", 1)
    .assign("total", 1, "aug", deps=['i', 'total'])
    .loop_iteration("i", 2)
    .assign("total", 3, "aug", deps=['i', 'total'])
    .loop_iteration("i", 3)
    .assign("total", 6, "aug", deps=['i', 'total'])
    .build()
)

compare(source_code, expected_trace, "LOOP_ITERATION Event")

## 6. WHILE_CONDITION - While Loops

While condition events with dependency tracking.

In [ ]:
# Create expected trace using DSL
source_code = """
x = 0
while x < 2:
    x += 1
"""

expected_trace = (
    trace()
    .assign("x", 0)
    .while_condition("x < 2", True, deps=['x'])
    .assign("x", 1, "aug", deps=['x'])
    .while_condition("x < 2", True, deps=['x'])
    .assign("x", 2, "aug", deps=['x'])
    .while_condition("x < 2", False, deps=['x'])
    .build()
)

compare(source_code, expected_trace, "WHILE_CONDITION Event")

## 7. ATTR_ASSIGN - Attribute Assignment

Object attribute assignment events.

In [ ]:
# Create expected trace using DSL
source_code = """
class Point:
    def __init__(self):
        self.x = 0
        self.y = 0

p = Point()
p.x = 10
"""

expected_trace = (
    trace()
    .function_entry("__init__", [])
    .assign("p.x", 0, "attr", obj_name="p", attr_name="x")
    .assign("p.y", 0, "attr", obj_name="p", attr_name="y")
    .return_event(None)
    .assign("p", None)  # Point object
    .assign("p.x", 10, "attr", obj_name="p", attr_name="x")
    .build()
)

compare(source_code, expected_trace, "ATTR_ASSIGN Event")

## 8. SUBSCRIPT_ASSIGN - Array/Dictionary Assignment

Subscript assignment events for arrays and dictionaries.

In [ ]:
# Create expected trace using DSL
source_code = """
arr = [1, 2, 3]
i = 1
arr[i] = 10
"""

expected_trace = (
    trace()
    .assign("arr", [1, 2, 3])
    .assign("i", 1)
    .assign("arr[i]", 10, "index", container_name="arr", index=1, deps=['i'])
    .build()
)

compare(source_code, expected_trace, "SUBSCRIPT_ASSIGN Event")

## 9. SLICE_ASSIGN - Slice Assignment

Slice assignment events for array slicing operations.

## 10. Complex Example - All EventTypes Together

A comprehensive example showing multiple EventTypes with dependency tracking.

In [ ]:
# Create expected trace using DSL
source_code = """
arr = [1, 2, 3, 4, 5]
start = 1
end = 3
arr[start:end] = [10, 20]
"""

expected_trace = (
    trace()
    .assign("arr", [1, 2, 3, 4, 5])
    .assign("start", 1)
    .assign("end", 3)
    .assign("arr[1:3]", [10, 20], "slice", container_name="arr", lower=1, upper=3, step=None, deps=['start', 'end'])
    .build()
)

compare(source_code, expected_trace, "SLICE_ASSIGN Event")

## Summary

This notebook demonstrates all EventTypes with integrated dependency tracking using the `deps` parameter:

1. **ASSIGN**: Variable assignments with `deps` showing which variables were read
2. **AUG_ASSIGN**: Augmented assignments (+=, -=, etc.) tracking both target and value dependencies  
3. **BRANCH**: Conditional branches with `deps` showing which variables influenced the condition
4. **FUNCTION_ENTRY/RETURN**: Function calls and returns
5. **LOOP_ITERATION**: For loop iterations
6. **WHILE_CONDITION**: While loop conditions with dependency tracking
7. **ATTR_ASSIGN**: Object attribute assignments
8. **SUBSCRIPT_ASSIGN**: Array/dictionary subscript assignments with index dependencies
9. **SLICE_ASSIGN**: Array slice assignments with slice bound dependencies

The `deps` parameter enables powerful dependency analysis by showing exactly which variables influenced each operation, enabling "Why did this happen?" questions to be answered effectively.

In [ ]:
# Create expected trace using DSL
source_code = """
def multiply(a, b):
    return a * b

x = 5
y = 3
result = multiply(x, y)

if result > 10:
    values = [result, x, y]
    values[0] = 100
"""

expected_trace = (
    trace()
    .assign("x", 5)
    .assign("y", 3)
    .function_entry("multiply", [5, 3])
    .return_event(15)
    .assign("result", 15)
    .branch("result > 10", True, "if_block", deps=['result'])
    .assign("values", [15, 5, 3], deps=['result', 'x', 'y'])
    .assign("values[0]", 100, "index", container_name="values", index=0)
    .build()
)

compare(source_code, expected_trace, "Complex Example - Multiple EventTypes")